In [1]:
import astral
from astral.sun import sun, sunrise, sunset
from astral.geocoder import database, lookup
from astral import LocationInfo
import datetime as dt
import math
import numpy as np
import psycopg2 as pg
import psycopg2.extras as pg_extras
from psycopg2.extensions import adapt, register_adapter, AsIs
import pytz
from IPython.display import clear_output

In [ ]:
conn = pg.connect('dbname=%s user=%s password=%s' % ('x2542', '<PG USERNAME>', '<PG PASSWORD>'))
conn.set_session(autocommit=True)

In [ ]:
ss_epoch = []
ss_yearly = []
years = []
lats = list(enumerate(np.linspace(-89, 89, 64)))
os = [LocationInfo(timezone='UTC', latitude=lat, longitude=0.0).observer for _, lat in lats]

In [ ]:
cur.execute('TRUNCATE locs CASCADE;')
for lat_i, lat in lats:
    cur.execute('INSERT INTO locs (id, lat, lng) VALUES (%s, %s, %s);', (lat_i, lat, 0))
cur.execute('TRUNCATE suns CASCADE;')

In [ ]:
class Range:
    def __init__(self, l, r, lex=False, rex=True):
        self.l = l
        self.r = r
        self.lex = lex
        self.rex = rex

def adapt_range(r):
    exb = ['[', '(']
    return AsIs("'%s%s, %s%s'" % ('(' if r.lex else '[', str(adapt(r.l.isoformat()).getquoted(), 'utf-8').strip('\''), str(adapt(r.r.isoformat()).getquoted(), 'utf-8').strip('\''), ')' if r.rex else ']')) # wowza that's a pain

register_adapter(Range, adapt_range)

# never mind use pg.extras.DateTimeRange

In [ ]:
last_fall = { lat_i: None for lat_i, _ in lats }
for nd in range((dt.date(2030, 1, 1) - dt.date(1970, 1, 1)).days):
    d = dt.date(1970, 1, 1) + dt.timedelta(days=nd)
#     if len(years) == 0 or d.year != years[-1]:
#         years.append(d.year)
    clear_output(wait=False)
    print(d)
    
    for (lat_i, lat), o in zip(lats, os):
        d_dt = dt.datetime.combine(d, dt.time())
        try:
            up = sunrise(o, date=d)
            down = sunset(o, date=d)
#             s += down - up # on second thought let postgres timedelta handle it. might have info on leap seconds and other shenanigans. do the update using window function cumsum grouping first by loc_id then (loc_id, year)
        except ValueError as e:
            if str(e) in ('Sun never reaches 6.0 degrees below the horizon, at this location.', 'Sun is always above the horizon on this day, at this location.'):
                up = d_dt # midnight this day
                down = d_dt + dt.timedelta(days=1) # midnight the next day (hopefully includes leap seconds)
#                 s += 86400
            if str(e) == 'Sun is always below the horizon on this day, at this location.':
                up = d_dt # midnight this day
                down = d_dt # midnight this day
        
        if last_fall[lat_i] != None:
            cur.execute('INSERT INTO suns (rise, falls, year, loc_id) VALUES (%s, %s::TSRANGE, %s, %s)', (up, Range(last_fall[lat_i], down, False, False), up.year, lat_i))
            
        last_fall[lat_i] = down